# Selenium https://rollbit.com/ trade scrape

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install webdriver-manager
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install pandas
# !{sys.executable} -m apt-get update 
# !{sys.executable} -m apt install chromium-chromedriver
!{sys.executable} -m pip install ipyparallel
!{sys.executable} -m pip install matplotlib

In [ ]:
import time
import datetime
import pandas as pd
import ipyparallel as ipp
from datetime import timezone
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup


In [ ]:
SLEEP = 0.5
ignored_exceptions = (StaleElementReferenceException,)
master_list = []

In [ ]:
def btn_click(xpath):
    try:        
        driver.execute_script("arguments[0].click();", WebDriverWait(
            driver, 1, SLEEP, ignored_exceptions=ignored_exceptions).until(
            EC.presence_of_element_located((By.XPATH, xpath))))
    except Exception as e:
        print(e)

In [ ]:
def update(html):
    _up = '#72f238'
    dt = datetime.datetime.now(timezone.utc)
    time = dt.strftime('%Y-%m-%d-%H:%M:%S.%f')[:-3]

    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.find('tbody').find_all('tr')

    for tr in trs:
        data = {}
        t = tr.find_all('td')
        
        data['time']         = time
        data['player']       = t[0].text
        data['trade']        = t[1].find('a')['href'].split('/')[2]
        _color               = t[1].find('svg', {'class': 'css-1pzuxf4'})['color']
        if _color == _up:
            data['position'] = 'UP'
        else:
            data['position'] = 'DOWN'
        data['wager']        = t[2].find('div').text
        data['multiplier']   = t[3].find('div').text
        data['pnl']          = t[4].find('div').text
        data['roi']          = t[5].find('div').text        

        print(data)
        master_list.append(data)        
        break   

In [ ]:
# SS5 = '74.208.76.181:1080'
options = Options()
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--proxy-server=socks5://' + SS5)
# driver = webdriver.Chrome('chromedriver',chrome_options=options)
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

In [ ]:
driver.get('https://rollbit.com')
# html = driver.page_source
# html

In [ ]:
_xpath_nav_trades = '//div[contains(text(),"Trades")]'
btn_click(_xpath_nav_trades)

In [ ]:
_c = 0
_prev = ''
_now = ''
_xpath_table = '//table'

while 1:
    try:
        html_table = driver.find_element(By.XPATH, _xpath_table).get_attribute('innerHTML')
        _now = hash(html_table)
        if _prev != _now:
            _prev = _now
            update(html_table)
            df = pd.DataFrame(master_list)
            %store df
    except:
        print('Something went wrong, WebDown or InternetDown?')
    
    _c += 1
    if _c > 50000: 
        print('end')
        break
#     time.sleep(0.1)

In [ ]:
df = pd.DataFrame(master_list)

In [ ]:
df

In [ ]:
df.to_csv('./data/sample_rollbit.csv', index=False)

In [ ]:
driver.quit()

# TEST

In [1]:
task_durations = [1] * 10
# request a cluster
with ipp.Cluster(n=4) as rc:
    # get a view on the cluster
    view = rc.load_balanced_view()
    # submit the tasks
    asyncresult = view.map_async(time.sleep, task_durations)
    # wait interactively for results
    asyncresult.wait_interactive()
    # retrieve actual results
    result = asyncresult.get()
# at this point, the cluster processes have been shutdown

NameError: name 'ipp' is not defined

In [ ]:
import gspread_dataframe as gd
import gspread as gs
gc = gs.service_account(filename="your/cred/file.json")

def export_to_sheets(worksheet_name,df,mode='r'):
    ws = gc.open("SHEET_NAME").worksheet("worksheet_name")
    if(mode=='w'):
        ws.clear()
        gd.set_with_dataframe(worksheet=ws,dataframe=df,include_index=False,include_column_header=True,resize=True)
        return True
    elif(mode=='a'):
        ws.add_rows(df.shape[0])
        gd.set_with_dataframe(worksheet=ws,dataframe=df,include_index=False,include_column_header=False,row=ws.row_count+1,resize=False)
        return True
    else:
        return gd.get_as_dataframe(worksheet=ws)
    
df = pd.DataFrame.from_records([{'a': i, 'b': i * 2} for i in range(100)])
export_to_sheets("SHEET_NAME",df,'a')